Importieren grundlegender Packages

In [94]:
# Dataframes und Datenanalyse-Funktionen
import pandas as pd
# Warnung für chained assignments ausschalten
pd.options.mode.chained_assignment = None

# Numerische Berechnungen mit Dataframes/Listen etc. 
import numpy as np

# "Simple" Diagramme
import matplotlib.pyplot as plt

# Komplexe Diagramme (exportierbar in html)
import seaborn as sns

# Einfache Klasse zum herunterladen von Google Drive Dateien
from google_drive_downloader import GoogleDriveDownloader as gdd

# Miscellaneous operating system interfaces
import os as os

# Basic date and time types
import datetime as dt

Definition des Chat-Dataframes

In [116]:
# Import der History.txt. Getrennt in 2 Spalten durch das ] Zeichen am Ende des Timestamps
# Spaltennamen als Timestamp und name_and_message definiert. Keine Kopfzeile.
# Text und Name muss noch getrennt werden. 
chat = pd.read_csv('Chatlog.txt', delimiter = ']', names = ['timestamp', 'name_and_message'], header = None)

# Leere Spalte für messages eingefügt
chat['message'] = ''

# Durchläuft den Chat von hinten nach vorne und prüft ob name_and_message ein missing value ist. Wenn ja, befindet sich die Nachricht im Timestamp-Feld und wird in das
# message Feld des vorherigen Eintrags eingetragen und dazu summiert. So Befindet sich die gesamte Nachricht beim zugehörigen Timestamp

for row_number in reversed(range(len(chat))):
    if pd.isna(chat.name_and_message[row_number]):
        chat['message'][row_number-1] =  chat.timestamp[row_number] + chat['message'][row_number]

# Die Zeilen mit missing values wurden so bearbeitet und können entfernt werden.

chat.dropna(inplace = True)
chat.reset_index(inplace = True, drop = True)

# Aufbereiten der timestamp-column zur Umwandlung des Objekts in ein Datetime-Objekt
# Entfernen des [ am Start des Timestamps

chat.timestamp.replace(regex = True, inplace = True, to_replace = r'\u200e', value = r'')

# Erkennen ob in timestamp ein Datum steht, das für einen reply spricht (Jahr und Sekunde fehlt).



chat['replied'] = chat.timestamp.str.contains('\[\d{2}\.\d+\.\, \d+\:\d+')
# Umwandlung zu Datetime

#chat['timestamp'] = pd.to_datetime(chat['timestamp'], format = '[%d.%m.%y, %H:%M:%S')

chat.tail()

,timestamp,name_and_message,message,replied
89995,"[27.04.19, 18:51:49",Tim Ackermann: ‎Bild weggelassen,,False
89996,"[27.04.19, 19:11:07",Yannic Büchele: ?,,False
89997,"[27.04.19, 19:22:46",emptytides: .,,False
89998,"[27.04.19, 19:26:15",Yannic Büchele: Echt so,,False
89999,"[27.04.19, 19:26:18",Yannic Büchele: Richtig was los hier,,False


In [117]:
chat['replied'] = chat.timestamp.str.contains('\[\d{2}\.\d+\.\, \d+\:\d+|\[\d{2}\:\d{2}\, \d+\/\d+\/\d{4}' )

# Anmerkung: Erkennt alle Timestamps als True ... WHY?!

for row_number in reversed(range(len(chat))):
    if chat['replied'][row_number] == True:
        chat['message'][row_number-1] = 'reply:' + chat.name_and_message[row_number]
        chat['replied'][row_number-1] = True

chat = chat[chat.timestamp.str.contains('\[\d{2}\.\d{2}\.\d{2}, \d{2}\:\d{2}\:\d{2}')]

chat['timestamp'] = pd.to_datetime(chat['timestamp'], format = '[%d.%m.%y, %H:%M:%S')